In [1]:
# !nvidia-smi

In [2]:

# !pip install mxnet
# !pip install gluonnlp pandas tqdm
# !pip install sentencepiece
# !pip install transformers==3
# !pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html


In [3]:
# !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [5]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [6]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [7]:
##GPU 사용 시
device = torch.device("cuda:0")

In [8]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [9]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [10]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [11]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [12]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [13]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [14]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [15]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [16]:
from google.colab import drive
isMount=!df |grep /content/drive
if len(isMount) < 1 : drive.mount('/content/drive')


In [17]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [18]:
# test_acc=0
# for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader2)):
#       token_ids = token_ids.long().to(device)
#       segment_ids = segment_ids.long().to(device)
#       valid_length= valid_length
#       label = label.long().to(device)
#       out = model(token_ids, valid_length, segment_ids)
#       test_acc += calc_accuracy(out, label)
# lable1=label.cpu().numpy()
# result=torch.max(out, 1)[1].cpu().numpy()
# print('-----------------')
# print("test_acc : ",test_acc)
# print("label  : ", lable1)
# print('result : ', result )



# train

In [96]:
## Setting parameters
max_len = 31
batch_size = 200
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [97]:
# dataset_train = nlp.data.TSVDataset("/content/drive/MyDrive/Lec_Capture/_dataNLP/news.tsv", field_indices=[1,2], num_discard_samples=1)
# dataset_train = nlp.data.TSVDataset("/content/drive/MyDrive/Lec_Capture/_dataNLP/WeMadeData.tsv", field_indices=[1,2], num_discard_samples=1)

dataset_train = nlp.data.TSVDataset("/content/drive/MyDrive/news_train.tsv", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("/content/drive/MyDrive/news_test.tsv", field_indices=[1,2], num_discard_samples=1)

In [98]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [99]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)


In [100]:
t_total = len(test_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [101]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/Lec_Capture/_dataNLP/myBertModel"))


<All keys matched successfully>

In [121]:

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


epoch 1 batch id 1 loss 0.001332966610789299 train acc 1.0

epoch 1 train acc 1.0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



epoch 1 test acc 0.7619047619047619


epoch 2 batch id 1 loss 0.0012582457857206464 train acc 1.0

epoch 2 train acc 1.0


...


epoch 8 test acc 0.7619047619047619


epoch 9 batch id 1 loss 0.0011825439287349582 train acc 1.0

epoch 9 train acc 1.0



epoch 9 test acc 0.7619047619047619


epoch 10 batch id 1 loss 0.0011651653330773115 train acc 1.0

epoch 10 train acc 1.0



epoch 10 test acc 0.7619047619047619


In [103]:
# # Model Save
# torch.save(model.state_dict(), "/content/drive/MyDrive/newsModel")


In [87]:
# model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
# model.load_state_dict(torch.load("/content/drive/MyDrive/newsModel"))


In [120]:
batch_size=21
dataset_test2 = nlp.data.TSVDataset("/content/drive/MyDrive/news_test.tsv", field_indices=[1,2], num_discard_samples=1)

data_test2 = BERTDataset(dataset_test2, 0, 1, tok, max_len, True, False)
test_dataloader2 = torch.utils.data.DataLoader(data_test2, batch_size=batch_size, num_workers=5)

t_total = len(test_dataloader2) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
# model.load_state_dict(torch.load("/content/drive/MyDrive/Lec_Capture/_dataNLP/myBertModel"))
# model.load_state_dict(torch.load("/content/drive/MyDrive/Lec_Capture/_dataNLP/newsModel"))


test_acc=0
label=0
result=0
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader2)):
      token_ids = token_ids.long().to(device)
      segment_ids = segment_ids.long().to(device)
      valid_length= valid_length
      label = label.long().to(device)
      out = model(token_ids, valid_length, segment_ids)
      test_acc += calc_accuracy(out, label)
      lable1=label.cpu().numpy()
      result=torch.max(out, 1)[1].cpu().numpy()
      print('-----------------')
      print("test_acc : ",test_acc)
      print("label  : ", lable1)
      print('result : ', result )



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


-----------------
test_acc :  0.7619047619047619
label  :  [1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]
result :  [1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 0 1 0]



In [109]:
!cat /content/drive/MyDrive/news_test.tsv

id	document	label
130	"유은혜, 교사들에게 손편지..""코로나 속 선생님들 노력에 감사"" 연합뉴스"	1
131	"잘못된 동선 공개에 눈물의 폐업, 업주 보상받을 길 열려 연합뉴스"	1
132	경기도내 어제 코로나19 확진 225명..사망자 10명 늘어 연합뉴스	1
133	"여수상의 ""변화·개혁 지적 수용..상공업 발전 최선"" 뉴스1"	1
134	"김한근 강릉시장, ""세계 속 강릉 향한 도약의 기틀 마련했다"" 뉴시스"	1
135	"""강원 '긴급생활안정조례' 전국 최초 발의"" vs ""레고랜드 임대수익 도의원도 몰라"" 노컷뉴스"	1
136	청주 교회발 연쇄 감염 '비상'..교인모임 뒤 10명 무더기 확진 연합뉴스	1
137	"""경찰, 고 박원순 성추행 내용 밝혀야 했는데 미진 .. 덮어선 안돼"" 머니투데이"	1
138	"법원 ""제자 성희롱·학대한 초등교사 파면은 정당"" 연합뉴스"	1
139	"부산 오시리아 테마파크 공사, 양대 노총 갈등에 몸살 연합뉴스"	1
140	"첫 확진자 후 346일만에 6만명..거리두기, 연장이냐 격상이냐 아시아경제"	1
167	"갤S21, 언팩 초대장까지 털렸다..""온라인 언팩 전에 안나온게 없네"" 뉴스1"	0
168	올해 모바일 앱 지출액 1천억 달러 돌파 '사상 최고' 지디넷코리아	0
169	"""갤버프부터 에어팟 맥스까지""..'음향기기' 시장 내년엔 더 뜨겁다 뉴스1"	0
170	"[2021 테크기상도] 코로나에도 선방한 반도체, 새해 본격 기지개 켠다 조선비즈"	0
171	어쩌면 코로나보다 더 무서울지 모릅니다 '기후변화 팬데믹' 경향신문	0
172	집콕族'에 훨훨 난 한국 게임..'산토끼' 마음까지 잡았다 서울경제	0
173	기후변화·코로나.. 인류는 새해에도 과학에 희망 건다 서울신문	0
174	속도 SKT·커버리지 LGU+·안정성 KT 1위 서울신문	0
175	네카라쿠배' 날았다..공인인증서·타다는 역사속으로 머니투데이	0
176	"LG 'QNED TV' 기습 발표에.. 삼성 ""